
# Laboratorio 2: Análisis de sensibilidad en problemas de Programación Lineal con Python

En este laboratorio, exploraremos el análisis de sensibilidad en problemas de Programación Lineal (PL) utilizando Python y la biblioteca Pyomo que ya introdujimos en la primera sesión de laboratorio. Como hemos aprendido, el **análisis de sensibilidad** es una herramienta que permite entender cómo los cambios en los parámetros de un modelo pueden afectar su solución óptima. 

En el contexto de la PL, este análisis se centra generalmente en dos aspectos:
1. Cambios en los **lados derechos de las restricciones** (es decir, en la disponibilidad de recursos).
2. Cambios en los **coeficientes de la función objetivo** (que representan el beneficio o costo de las variables de decisión).

Aunque Pyomo no cuenta con herramientas específicas para realizar análisis de sensibilidad directamente, podemos implementarlo modificando los parámetros del modelo y observando el efecto en la solución.

Pyomo permite resolver problemas de programación lineal, pero no tiene funcionalidades nativas específicas para realizar análisis de sensibilidad como las que podrías encontrar en paquetes más especializados en optimización, como Gurobi o CPLEX. Sin embargo, existen algunas estrategias que puedes usar para obtener información sobre el análisis de sensibilidad de un  problema de PL. 

Por otro lado, GLPK, que es el solver que introdujimos en el Laboratorio 1, es una herramienta robusta para resolver problemas de programación lineal y entera, pero, desafortunadamente, tiene limitaciones para realizar análisis de sensibilidad, ya que no proporciona directamente precios sombra ni rangos de factibilidad en su salida. Sin embargo, podemos llevar a cabo una estrategia de análisis de sensibilidad manual con GLPK en Pyomo.




## Análisis de sensibilidad sobre los lados derechos de las restricciones: cálculos de precios sombra

El análisis de sensibilidad sobre **los lados derechos de las restricciones** permite determinar cómo cambia la solución óptima cuando varían los límites de las restricciones, tales como la disponibilidad de recursos o capacidades. Esto es especialmente útil para entender la robustez de una solución ante cambios en la disponibilidad de recursos. GLPK puede proporcionar valores duales en ciertos problemas de programación lineal.

1. **Configura el modelo en Pyomo** tal como lo harías normalmente, definiendo las restricciones y la función objetivo.
2. **Activa la opción de valores duales** en Pyomo añadiendo un `Suffix` al modelo:
   ```python
   model.dual = Suffix(direction=Suffix.IMPORT)
   ```
   En Pyomo, un Suffix es una estructura que permite asociar datos adicionales a componentes de un modelo, como restricciones o variables, y se utiliza comúnmente para trabajar con información complementaria que generan los solucionadores, como los valores duales (o precios sombra) en problemas de optimización lineal.

   ¿Para qué sirve Suffix?

   1. Acceso a valores duales: En problemas de optimización lineal, los solucionadores pueden calcular valores duales para las restricciones activas. Los valores duales indican el cambio en el valor de la función objetivo si se incrementa el límite de una restricción en una unidad. Estos valores son fundamentales para el análisis de sensibilidad.

   2. Datos adicionales del solucionador: Algunos solucionadores generan datos adicionales (como los precios sombra, el estado de las restricciones, u otras métricas de diagnóstico) que se pueden capturar y almacenar usando `Suffix`.

   Sintaxis de Suffix

   Para crear un `Suffix`, se usa la siguiente sintaxis:
   ```python
   model.suffix_name = Suffix(direction=Suffix.IMPORT)
   ```

   Aquí:
   - `model.suffix_name`: es el nombre que asignas al sufijo, por ejemplo, `model.dual`.
   - `direction=Suffix.IMPORT`: indica que el sufijo se importa desde el solucionador al modelo en Pyomo. Esto significa que Pyomo llenará el sufijo con datos provenientes del solucionador. También existen otros valores posibles para `direction`, como `Suffix.EXPORT`, que permite enviar datos desde Pyomo al solucionador.

3. **Extrae los valores duales** después de resolver el modelo con GLPK. Los valores duales estarán disponibles en `model.dual[constraint]`, siempre que el problema sea lineal y GLPK pueda calcularlos.

### Ejemplo

Aquí tienes una adaptación de un ejemplo básico para un modelo de producción que utiliza GLPK y extrae los valores duales para realizar un análisis de sensibilidad:



In [1]:
from pyomo.environ import *

# Crear el modelo
model = ConcreteModel()

# Definir variables de decisión
model.x = Var(within=NonNegativeReals)
model.y = Var(within=NonNegativeReals)

# Definir función objetivo
model.obj = Objective(expr=40*model.x + 30*model.y, sense=maximize)

# Definir restricciones
model.demand = Constraint(expr=model.x <= 40)
model.laborA = Constraint(expr=model.x + model.y <= 80)
model.laborB = Constraint(expr=2*model.x + model.y <= 100)

# Añadir el sufijo dual para obtener los valores duales
model.dual = Suffix(direction=Suffix.IMPORT)

# Resolver el modelo usando GLPK
solver = SolverFactory('glpk')
solver.solve(model)

# Imprimir la solución
print("x =", model.x())
print("y =", model.y())
print("Valor de la función objetivo =", model.obj())

# Realizar el análisis de sensibilidad con los valores duales
print("\nAnálisis de sensibilidad:")
print("Dual de la restricción de demanda:", model.dual[model.demand])
print("Dual de la restricción de laborA:", model.dual[model.laborA])
print("Dual de la restricción de laborB:", model.dual[model.laborB])


x = 20.0
y = 60.0
Valor de la función objetivo = 2600.0

Análisis de sensibilidad:
Dual de la restricción de demanda: 0.0
Dual de la restricción de laborA: 20.0
Dual de la restricción de laborB: 10.0


### Limitaciones
Es importante señalar que GLPK solo proporciona valores duales en problemas lineales y no siempre tiene la capacidad de devolver información tan completa como otros solucionadores, como Gurobi o CPLEX. Sin embargo, para análisis de sensibilidad básicos, como el cálculo de precios sombra, es suficiente.


## Análisis de sensibilidad sobre los coeficientes de la función objetivo

También es posible realizar análisis de sensibilidad sobre los **coeficientes de la función objetivo** en un problema de programación lineal usando el solver GLPK y la herramienta de formulación de problemas Pyomo. Esto implica analizar cómo varía el valor óptimo y la solución del modelo cuando se modifican los coeficientes de la función objetivo. Aunque Pyomo no ofrece un análisis de sensibilidad de estos coeficientes directamente, hay maneras de hacerlo manualmente.

### Procedimiento para realizar el análisis de sensibilidad de los coeficientes de la función objetivo

Para analizar cómo afectan los cambios en los coeficientes de la función objetivo a la solución óptima, puedes seguir estos pasos:

1. **Resolver el modelo base**: Primero, resuelve el modelo con los coeficientes originales de la función objetivo y guarda la solución óptima y el valor de la función objetivo.

2. **Modificar los coeficientes de la función objetivo**: Incrementa o disminuye cada coeficiente de la función objetivo de manera controlada usando bucles. Puedes hacerlo en un rango de valores para observar cómo afecta cada cambio a la solución óptima y al valor de la función objetivo.

3. **Re-optimizar el modelo**: Resuelve el modelo nuevamente después de cada modificación de un coeficiente. Registra el nuevo valor de la función objetivo y la nueva solución.

4. **Analizar los resultados**: Compara los resultados obtenidos en cada iteración con los de la solución base. Esto te dará una idea de la sensibilidad de la solución óptima respecto a los cambios en los coeficientes de la función objetivo.

### Ejemplo

Aquí tienes un ejemplo sencillo de cómo hacer esto con Pyomo:


In [2]:
from pyomo.environ import *

# Crear el modelo
model = ConcreteModel()

# Definir variables de decisión
model.x = Var(within=NonNegativeReals)
model.y = Var(within=NonNegativeReals)

# Función objetivo inicial
model.obj = Objective(expr=40 * model.x + 30 * model.y, sense=maximize)

# Definir restricciones
model.demand = Constraint(expr=model.x <= 40)
model.laborA = Constraint(expr=model.x + model.y <= 80)
model.laborB = Constraint(expr=2 * model.x + model.y <= 100)

# Resolver el modelo base
solver = SolverFactory('glpk')
solver.solve(model)

# Guardar los valores iniciales
original_obj_value = model.obj()
original_x = model.x()
original_y = model.y()

print("Solución base:")
print("Valor objetivo:", original_obj_value)
print("x =", original_x)
print("y =", original_y)

# Análisis de sensibilidad sobre el coeficiente de x en la función objetivo
coef_range = range(30, 51, 5)  # Cambiar el coeficiente de 30 a 50 en pasos de 5
sens_results = []

for coef_x in coef_range:
    # Actualizar el coeficiente de x en la función objetivo
    model.obj.set_value(coef_x * model.x + 30 * model.y)
    
    # Resolver el modelo modificado
    solver.solve(model)
    
    # Guardar los resultados
    sens_results.append({
        'coef_x': coef_x,
        'obj_value': model.obj(),
        'x': model.x(),
        'y': model.y()
    })

# Mostrar resultados de sensibilidad
print("\nAnálisis de sensibilidad en el coeficiente de x en la función objetivo:")
for result in sens_results:
    print(f"Coeficiente de x: {result['coef_x']}, Valor objetivo: {result['obj_value']}, x = {result['x']}, y = {result['y']}")

Solución base:
Valor objetivo: 2600.0
x = 20.0
y = 60.0

Análisis de sensibilidad en el coeficiente de x en la función objetivo:
Coeficiente de x: 30, Valor objetivo: 2400.0, x = 20.0, y = 60.0
Coeficiente de x: 35, Valor objetivo: 2500.0, x = 20.0, y = 60.0
Coeficiente de x: 40, Valor objetivo: 2600.0, x = 20.0, y = 60.0
Coeficiente de x: 45, Valor objetivo: 2700.0, x = 20.0, y = 60.0
Coeficiente de x: 50, Valor objetivo: 2800.0, x = 20.0, y = 60.0


### Limitaciones

Este análisis manual tiene algunas limitaciones:
- Es computacionalmente costoso si tienes muchos coeficientes o si deseas evaluar múltiples valores de cambio.
- No proporciona rangos exactos (como el análisis de rango) para los coeficientes en los que la solución óptima se mantiene constante.

Para obtener un análisis de rango exacto de los coeficientes de la función objetivo, algunos solucionadores avanzados (como CPLEX y Gurobi) ofrecen esta funcionalidad de manera automática. Sin embargo, en Pyomo, un análisis de sensibilidad manual es una buena aproximación y puede ser suficiente para entender cómo afectan los cambios en los coeficientes a la solución del problema.

## Ejercicios de análisis de sensibilidad en problemas de PL con Pyomo

#### Ejercicio 1

Una empresa fabrica tres productos diferentes: producto A, producto B y producto C. Para su fabricación, la empresa emplea cuatro recursos limitados: horas de trabajo de las máquinas, materia prima, capacidad de almacenamiento y horas de trabajo de los empleados. La disponibilidad mensual de cada recurso se presenta a continuación:

- Horas de trabajo de las máquinas: 600 horas.
- Materia prima: 500 kg.
- Capacidad de almacenamiento: 400 unidades.
- Horas de trabajo de empleados: 300 horas.

La siguiente tabla muestra la cantidad de recursos que se necesitan para producir una unidad de cada producto:

| Producto | Horas de máquinas | Materia prima | Almacenamiento | Horas de empleados |
|----------|--------------------|---------------|----------------|---------------------|
| A        | 8                 | 4             | 2              | 3                   |
| B        | 5                 | 6             | 4              | 2                   |
| C        | 7                 | 5             | 3              | 5                   |

El beneficio por cada unidad fabricada del producto A es de 40 €, del producto B es de 50 €, y del producto C es de 60 €. La empresa desea maximizar el beneficio total de su producción, dadas las limitaciones de recursos disponibles.

Se pide:

1. Formule el problema de producción óptima como un problema de Programación Lineal y resuélvalo.
2. Calcule los precios sombra sin resolver el problema dual e interprete los precios sombra obtenidos.
3. Formule el problema dual asociado al problema de producción óptima, resuélvalo y compare con las soluciones obtenidas en el apartado anterior.
4. Trate de determinar cuánto puede variar el beneficio unitario del producto A sin que varíe la solución óptima del problema primal. ¿Y el beneficio unitario del producto B?





In [ ]:
from pyomo.environ import *

model = ConcreteModel()

model.x_A = Var(within=NonNegativeReals)
model.x_B = Var(within=NonNegativeReals)
model.x_C = Var(within=NonNegativeReals)

model.obj = Objective(expr=40*model.x_A + 50*model.x_B + 60*model.x_C, sense=maximize)

model.horas = Constraint(expr=8 *model.x_A + 5*model.x_B + 7*model.x_C <= 600)
model.materia= Constraint(expr=4*model.x_A + 6*model.x_B + 5*model.x_C <=500)
model.capacidad = Constraint(expr=2*model.x_A + 4*model.x_B + 3*model.x_C <= 400)
model.empleados = Constraint(expr=3*model.x_A + 2*model.x_B + 5*model.x_C <= 300)

model.dual = Suffix(direction=Suffix.IMPORT)

solver = SolverFactory('glpk')
solver.solve(model)

print(f"Producción óptima de A: {model.x_A.value}")
print(f"Producción óptima de B: {model.x_B.value}")
print(f"Producción óptima de C: {model.x_C.value}")
print(f"Ganancia máxima: {model.obj()}")

print("\nPrecios Sombra (Valores duales):")
print(f"Precio sombra restricción de horas de máquina: {model.dual[model.horas]}")
print(f"Precio sombra restricción de materia prima: {model.dual[model.materia]}")
print(f"Precio sombra restricción de capacidad de almacenamiento: {model.dual[model.capacidad]}")
print(f"Precio sombra restricción de horas de empleados: {model.dual[model.empleados]}")

# Análisis de sensibilidad

# Definir el rango de coeficientes para el análisis de sensibilidad de x_A
coef_range_A = range(35, 61, 5)  # Cambiar el beneficio unitario de A de 35 a 60 en pasos de 5
sens_results_A = []

for coef_A in coef_range_A:
    # Actualizar el coeficiente de x_A en la función objetivo
    model.obj.set_value(coef_A * model.x_A + 50 * model.x_B + 60 * model.x_C)
    
    # Resolver el modelo modificado
    solver.solve(model)
    
    # Guardar los resultados
    sens_results_A.append({
        'coef_A': coef_A,
        'obj_value': model.obj(),
        'x_A': model.x_A(),
        'x_B': model.x_B(),
        'x_C': model.x_C()
    })

# Mostrar resultados de sensibilidad para el beneficio de x_A
print("\nAnálisis de sensibilidad en el beneficio unitario de A en la función objetivo:")
for result in sens_results_A:
    print(f"Coeficiente de A: {result['coef_A']}, Valor objetivo: {result['obj_value']}, x_A = {result['x_A']}, x_B = {result['x_B']}, x_C = {result['x_C']}")

# Definir el rango de coeficientes para el análisis de sensibilidad de x_B
coef_range_B = range(45, 66, 5)  # Cambiar el beneficio unitario de B de 45 a 65 en pasos de 5
sens_results_B = []

for coef_B in coef_range_B:
    # Actualizar el coeficiente de x_B en la función objetivo
    model.obj.set_value(40 * model.x_A + coef_B * model.x_B + 60 * model.x_C)
    
    # Resolver el modelo modificado
    solver.solve(model)
    
    # Guardar los resultados
    sens_results_B.append({
        'coef_B': coef_B,
        'obj_value': model.obj(),
        'x_A': model.x_A(),
        'x_B': model.x_B(),
        'x_C': model.x_C()
    })

# Mostrar resultados de sensibilidad para el beneficio de x_B
print("\nAnálisis de sensibilidad en el beneficio unitario de B en la función objetivo:")
for result in sens_results_B:
    print(f"Coeficiente de B: {result['coef_B']}, Valor objetivo: {result['obj_value']}, x_A = {result['x_A']}, x_B = {result['x_B']}, x_C = {result['x_C']}")




Producción óptima de A: 0.0
Producción óptima de B: 50.0
Producción óptima de C: 40.0
Ganancia máxima: 4900.0

Precios Sombra (Valores duales):
Precio sombra restricción de horas de máquina: 0.0
Precio sombra restricción de materia prima: 6.5
Precio sombra restricción de capacidad de almacenamiento: 0.0
Precio sombra restricción de horas de empleados: 5.5

Análisis de sensibilidad en el beneficio unitario de A en la función objetivo:
Coeficiente de A: 35, Valor objetivo: 4900.0, x_A = 0.0, x_B = 50.0, x_C = 40.0
Coeficiente de A: 40, Valor objetivo: 4900.0, x_A = 0.0, x_B = 50.0, x_C = 40.0
Coeficiente de A: 45, Valor objetivo: 4953.84615384615, x_A = 21.5384615384615, x_B = 44.6153846153846, x_C = 29.2307692307692
Coeficiente de A: 50, Valor objetivo: 5061.538461538457, x_A = 21.5384615384615, x_B = 44.6153846153846, x_C = 29.2307692307692
Coeficiente de A: 55, Valor objetivo: 5169.230769230764, x_A = 21.5384615384615, x_B = 44.6153846153846, x_C = 29.2307692307692
Coeficiente de A: 6

Este análisis de sensibilidad permite entender cómo los cambios en los beneficios unitarios de 
𝐴
A y 
𝐵
B afectan la estrategia de producción. Específicamente:

Incrementos en el beneficio de 
𝐴
A pueden hacer que sea rentable producirlo, afectando la mezcla de productos.
El beneficio de 
𝐵
B, aunque impacta la ganancia total, no altera significativamente la estrategia de producción en el rango analizado.
Los precios sombra ayudan a identificar cuáles recursos adicionales serían valiosos para mejorar la ganancia total.
Estos insights permiten tomar decisiones estratégicas sobre dónde enfocar los esfuerzos para aumentar la rentabilidad, ya sea ajustando precios de venta o invirtiendo en los recursos que limitan la producción.

### Ejercicio 2

Un médico ha recetado a una paciente una pauta alimenticia que se basa en tres alimentos: arroz, pescado y verduras frescas. La combinación de estos alimentos debe cumplir con unos requisitos mínimos de nutrientes, concretamente en términos de proteínas y calorías. Los requisitos mínimos son:

- Proteínas: al menos 3 unidades.
- Calorías: al menos 4,000 calorías.

Cada alimento aporta una cantidad específica de proteínas y calorías por kilogramo, como se muestra en la siguiente tabla:

| Alimento          | Proteínas (unidades/kg) | Calorías (kcal/kg) | Coste (unidades monetarias/kg) |
|-------------------|-------------------------|---------------------|--------------------------------|
| Arroz             | 1                       | 2,000              | 55                             |
| Pescado           | 3                       | 3,000              | 125                            |
| Verduras frescas  | 2                       | 1,000              | 55                             |

La paciente desea diseñar una dieta que minimice el coste total, respetando los requisitos de nutrientes impuestos por el médico.

Se pide:

1. Formule el problema de optimización de la dieta como un problema de Programación Lineal que minimice el coste de los alimentos mientras se cumplen los requisitos de nutrientes.

2. Resuelva el problema de Programación Lineal para determinar la cantidad óptima de cada alimento en la dieta.

3. Analice cómo afecta al coste total un aumento del 10% en el coste del arroz. ¿Se modifica la solución óptima del problema primal con este cambio?

4. Formule el problema dual asociado a la optimización de la dieta y resuélvalo. Interprete los precios sombra obtenidos y explique su significado en el contexto de este problema. ¿Qué implicaciones tiene un precio sombra positivo en los requisitos de nutrientes?


In [20]:
from pyomo.environ import *

model = ConcreteModel()

model.arroz = Var(within=NonNegativeReals)
model.pescado = Var(within=NonNegativeReals)
model.verdura = Var(within=NonNegativeReals)

model.obj = Objective(expr=55*model.arroz + 125*model.pescado + 55*model.verdura, sense=minimize)

model.proteinas = Constraint(expr=1*model.arroz + 3*model.pescado + 2*model.verdura >= 3)
model.calorias = Constraint(expr=2*model.arroz + 3*model.pescado + 1*model.verdura >= 4)

model.dual = Suffix(direction=Suffix.IMPORT)

solver = SolverFactory('glpk')
solver.solve(model)

print(f"Cantidad óptima de arroz: {model.arroz():.2f} kg")
print(f"Cantidad óptima de pescado: {model.pescado():.2f} kg")
print(f"Cantidad óptima de verduras: {model.verdura():.2f} kg")
print(f"Coste mínimo de la dieta: {model.obj():.2f} euros")

coste_arroz = 55
print("\n--- Análisis de sensibilidad: aumento del 10% n el coste del arroz ---")
coste_arroz_nuevo = coste_arroz * 1.1

model.obj.set_value(coste_arroz_nuevo*model.arroz + 125*model.pescado + 55*model.verdura)
# Resultados tras el incremento del 10% en el coste del arroz
print(f"Nuevo coste mínimo con el 10% de aumento en el coste del arroz: {model.obj():.2f} euros")
print(f"Cantidad de arroz en la nueva solución: {model.arroz():.2f} kg")
print(f"Cantidad de pescado en la nueva solución: {model.pescado():.2f} kg")
print(f"Cantidad de verduras en la nueva solución: {model.verdura():.2f} kg")


proteinas_dual = model.dual[model.proteinas]
calorias_dual = model.dual[model.calorias]

print(f"Precio sombra de la restricción de proteínas: {proteinas_dual:.2f}")
print(f"Precio sombra de la restricción de calorías: {calorias_dual:.2f}")

# Interpretación de los precios sombra
if proteinas_dual > 0:
    print("La restricción de proteínas es activa. Aumentar el requisito de proteínas incrementaría el coste total.")
else:
    print("La restricción de proteínas no es activa. Incrementar la disponibilidad de proteínas no afectará el coste total.")

if calorias_dual > 0:
    print("La restricción de calorías es activa. Aumentar el requisito de calorías incrementaría el coste total.")
else:
    print("La restricción de calorías no es activa. Incrementar la disponibilidad de calorías no afectará el coste total.")





Cantidad óptima de arroz: 1.67 kg
Cantidad óptima de pescado: 0.00 kg
Cantidad óptima de verduras: 0.67 kg
Coste mínimo de la dieta: 128.33 euros

--- Análisis de sensibilidad: aumento del 10% n el coste del arroz ---
Nuevo coste mínimo con el 10% de aumento en el coste del arroz: 137.50 euros
Cantidad de arroz en la nueva solución: 1.67 kg
Cantidad de pescado en la nueva solución: 0.00 kg
Cantidad de verduras en la nueva solución: 0.67 kg
Precio sombra de la restricción de proteínas: 18.33
Precio sombra de la restricción de calorías: 18.33
La restricción de proteínas es activa. Aumentar el requisito de proteínas incrementaría el coste total.
La restricción de calorías es activa. Aumentar el requisito de calorías incrementaría el coste total.
